#Importing Libraries

##For Web Scraping

In [2]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
import numpy as np

#Web Scraping

###Scraping The Data And Getting All URLs In A LIst

In [3]:
urls = []
with open('url_file.csv') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
      urls.append(row["URL"])

###Getting the Content From All URLs

In [4]:
context = []

for i in urls:
  response = requests.get(i)
  soup = BeautifulSoup(response.content,'lxml')
  find_div = soup.find('div',class_= "td-post-content")
  if find_div is not None:
    context.append(find_div.text)
  else:
    context.append("Not Available")  

#Creating The DataFrame

In [5]:
data = zip(urls,context)

df = pd.DataFrame(data,columns=["Websites","Context"])
df

,Websites,Context
0,https://insights.blackcoffer.com/ai-in-healthc...,\nIntroduction\n“If anything kills over 10 mil...
1,https://insights.blackcoffer.com/what-if-the-c...,"\nHuman minds, a fascination in itself carryin..."
2,https://insights.blackcoffer.com/what-jobs-wil...,\nIntroduction\nAI is rapidly evolving in the ...
3,https://insights.blackcoffer.com/will-machine-...,\n“Anything that could give rise to smarter-th...
4,https://insights.blackcoffer.com/will-ai-repla...,\n“Machine intelligence is the last invention ...
...,...,...
109,https://insights.blackcoffer.com/blockchain-fo...,\nReconciling with the financial realities of ...
110,https://insights.blackcoffer.com/the-future-of...,\nWhat Is an Investment?\nAn investment is a r...
111,https://insights.blackcoffer.com/big-data-anal...,\nQuality and affordable healthcare is a visio...
112,https://insights.blackcoffer.com/business-anal...,\nAnalytics is a statistical scientific proces...


###Replacing the \n from the context

In [6]:
df['Context'] = df['Context'].replace(r'\n',' ', regex=True) 

In [7]:
df

,Websites,Context
0,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...
1,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying..."
2,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...
3,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...
4,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...
...,...,...
109,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...
110,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...
111,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...
112,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...


#Importing Libraries

## For Sentiment Analysis

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

#warnings
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [9]:
stop_words = set(stopwords.words('english'))
print(stop_words)

{'your', 'into', 'does', "it's", 'an', "you'll", 'needn', 'during', 'but', 'themselves', 'its', 'then', 'now', 'below', 'up', 'most', 'there', 'theirs', 'at', 'by', 'in', 'aren', 'mightn', 'these', 'all', 'shouldn', 'more', 'shan', 'you', 'as', 'no', 'didn', "couldn't", 'he', 'once', 'hadn', 'itself', 'ain', 'are', 'whom', 'under', 'those', 'above', 'were', 'than', 'some', 'hers', 'if', 'when', 'so', 'we', 'few', "hadn't", 'yourself', 'or', 'out', 'with', 'here', 'ourselves', 'very', 'the', "needn't", 'have', 'yours', 'own', 'while', 'was', 'll', "mightn't", 'what', 'that', 'this', 'a', 'each', 'is', 'yourselves', "wasn't", 'down', 'why', 'until', 'before', 'off', "shouldn't", 'wasn', 'them', 't', 'against', 'over', 'further', 'myself', 'been', "should've", "haven't", 'herself', 'it', 'my', 'how', 'of', 'wouldn', 'y', 'too', "you're", 'both', 'they', 'won', 'again', 'through', 'just', 'which', 'm', 'such', 'ours', 'because', 'o', 'any', 'do', 'she', 'on', 'isn', "shan't", 'ma', 'don', 

In [10]:
example = df['Context'][0]
tokens = word_tokenize(example)
tokens

['Introduction',
 '“',
 'If',
 'anything',
 'kills',
 'over',
 '10',
 'million',
 'people',
 'in',
 'the',
 'next',
 'few',
 'decades',
 ',',
 'it',
 'will',
 'be',
 'a',
 'highly',
 'infectious',
 'virus',
 'rather',
 'than',
 'a',
 'war',
 '.',
 'Not',
 'missiles',
 'but',
 'microbes.',
 '”',
 'Bill',
 'Gates',
 '’',
 's',
 'remarks',
 'at',
 'a',
 'TED',
 'conference',
 'in',
 '2014',
 ',',
 'right',
 'after',
 'the',
 'world',
 'had',
 'avoided',
 'the',
 'Ebola',
 'outbreak',
 '.',
 'When',
 'the',
 'new',
 ',',
 'unprecedented',
 ',',
 'invisible',
 'virus',
 'hit',
 'us',
 ',',
 'it',
 'met',
 'an',
 'overwhelmed',
 'and',
 'unprepared',
 'healthcare',
 'system',
 'and',
 'oblivious',
 'population',
 '.',
 'This',
 'public',
 'health',
 'emergency',
 'demonstrated',
 'our',
 'lack',
 'of',
 'scientific',
 'consideration',
 'and',
 'underlined',
 'the',
 'alarming',
 'need',
 'for',
 'robust',
 'innovations',
 'in',
 'our',
 'health',
 'and',
 'medical',
 'facilities',
 '.',
 'Fo

In [11]:
tokenize_words_without_stopwords = []

for word in tokens:
  if word not in stop_words:
    tokenize_words_without_stopwords.append(word)
print(tokenize_words_without_stopwords)    

['Introduction', '“', 'If', 'anything', 'kills', '10', 'million', 'people', 'next', 'decades', ',', 'highly', 'infectious', 'virus', 'rather', 'war', '.', 'Not', 'missiles', 'microbes.', '”', 'Bill', 'Gates', '’', 'remarks', 'TED', 'conference', '2014', ',', 'right', 'world', 'avoided', 'Ebola', 'outbreak', '.', 'When', 'new', ',', 'unprecedented', ',', 'invisible', 'virus', 'hit', 'us', ',', 'met', 'overwhelmed', 'unprepared', 'healthcare', 'system', 'oblivious', 'population', '.', 'This', 'public', 'health', 'emergency', 'demonstrated', 'lack', 'scientific', 'consideration', 'underlined', 'alarming', 'need', 'robust', 'innovations', 'health', 'medical', 'facilities', '.', 'For', 'past', 'years', ',', 'artificial', 'intelligence', 'proven', 'tangible', 'potential', 'healthcare', 'sectors', ',', 'clinical', 'practices', ',', 'translational', 'medical', 'biomedical', 'research', '.', 'After', 'first', 'case', 'detected', 'China', 'December', '31st', '2019', ',', 'AI', 'program', 'develo

In [12]:
def remove_stopwords():
  for sentense in list(df['Context']):
    tokens = word_tokenize(sentense)
    tokenize_words_without_stopwords = []
    for word in tokens:
      if word not in stop_words:
        tokenize_words_without_stopwords.append(word)
    return tokenize_words_without_stopwords

###Calculating word count after removing stop words

In [13]:
count_word = []
for i in df['Context']:
  tokens = word_tokenize(i)
  tokenize_words_without_stopwords = []
  for word in tokens:
    if word not in stop_words:
      tokenize_words_without_stopwords.append(word)
  count_word.append(len(tokenize_words_without_stopwords))

In [14]:
df["Count of words"] = count_word

###Creating Ids

In [15]:
df = df.reset_index()
df = df.rename(columns={'index':'Ids'})
df

,Ids,Websites,Context,Count of words
0,0,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396
1,1,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012
2,2,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255
3,3,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157
4,4,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253
...,...,...,...,...
109,109,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672
110,110,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212
111,111,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847
112,112,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516


###Creating sentiment analyzer

In [16]:
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm.notebook import tqdm
nltk.download('vader_lexicon')

sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


In [17]:
sia.polarity_scores(example) ##Checking example

{'neg': 0.044, 'neu': 0.811, 'pos': 0.145, 'compound': 0.9996}

###Applying the analyzer on entire dataset

In [18]:
# Run the polarity score on the entire dataset
res = {}
for i, row in tqdm(df.iterrows(), total=len(df)):
    text = row['Context']
    myid = row['Ids']
    res[myid] = sia.polarity_scores(text)

  0%|          | 0/114 [00:00<?, ?it/s]

###Merging the result with dataset

In [19]:
vaders = pd.DataFrame(res).T
vaders = vaders.reset_index().rename(columns={'index': 'Ids'})
vaders = vaders.merge(df, how='left')

###Renaming the dataset

In [20]:
df = vaders.iloc[:,:]
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253
...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516


##Polarity Score

In [21]:
polarity_score = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    pos = row['pos']
    neg = row['neg']
    polarity = (pos - neg)/ ((pos + neg) + 0.000001)
    polarity_score.append(polarity)

  0%|          | 0/114 [00:00<?, ?it/s]

In [22]:
df["Polarity Score"] = polarity_score
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383
...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018


##Subjectivity Score

In [23]:
subjectivity_score = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    pos = row['pos']
    neg = row['neg']
    total_word = row['Count of words']
    sub = (pos + neg)/ ((total_word) + 0.000001)
    subjectivity_score.append(sub)

  0%|          | 0/114 [00:00<?, ?it/s]

In [24]:
df["Subjectivity Score"] = subjectivity_score
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157
...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337


###Importing libraries for sentense count

In [25]:
from nltk.tokenize import sent_tokenize

In [26]:
texts = df['Context']
count_sentense = []
for text in texts:
  total_sentense = sent_tokenize(text)
  count_sentense.append(len(total_sentense))

In [27]:
df['number_sentense'] = count_sentense

##Average Sentense Length

In [28]:
average_sentense_length = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    number_words = row['Count of words']
    number_sentense = row['number_sentense']
    sentense_length = round((number_words / number_sentense),2)
    average_sentense_length.append(sentense_length)

  0%|          | 0/114 [00:00<?, ?it/s]

In [29]:
df['Average Sentense Length'] = average_sentense_length
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230,80,12.65
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088,85,14.76
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163,95,12.18
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157,79,15.86
...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268,49,13.71
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107,62,19.55
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216,66,12.83
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337,29,17.79


###Importing libraries for syllable word count

In [32]:
import spacy
from textstat.textstat import textstatistics

##Complex word count

In [33]:
complex_word_count = []

for i, row in tqdm(df.iterrows(), total=len(df)):
  words = []
  text = row['Context']
  tokens = word_tokenize(text)
  
  for word in tokens:
    if word not in stop_words:
      words.append(word)
  
  diff_words_set = []
  nlp = spacy.load('en_core_web_sm')

  for word in words:
    syllable_count = textstatistics().syllable_count(word)
    if word not in nlp.Defaults.stop_words and syllable_count >= 2:
      diff_words_set.append(word)
  complex_word_count.append(len(diff_words_set))

  0%|          | 0/114 [00:00<?, ?it/s]

In [34]:
df['Complex Word Count'] = complex_word_count
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length,Complex Word Count
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13,798
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230,80,12.65,397
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088,85,14.76,707
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163,95,12.18,513
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157,79,15.86,627
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268,49,13.71,331
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107,62,19.55,559
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216,66,12.83,460
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337,29,17.79,308


##Percentage of complex words

In [35]:
percentage_complex_word = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    number_words = row['Count of words']
    complex_word = row['Complex Word Count']
    percentage = round((complex_word / number_words) * 100,2)
    percentage_complex_word.append(percentage)

  0%|          | 0/114 [00:00<?, ?it/s]

In [36]:
df["Percentage of Complex words"] = percentage_complex_word
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length,Complex Word Count,Percentage of Complex words
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13,798,57.16
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230,80,12.65,397,39.23
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088,85,14.76,707,56.33
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163,95,12.18,513,44.34
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157,79,15.86,627,50.04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268,49,13.71,331,49.26
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107,62,19.55,559,46.12
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216,66,12.83,460,54.31
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337,29,17.79,308,59.69


## Fog index

In [37]:
f_index = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    sentense_length = row['Average Sentense Length']
    complex_word = row['Percentage of Complex words']
    fog_index = 0.4 * (sentense_length + complex_word)
    f_index.append(fog_index)

  0%|          | 0/114 [00:00<?, ?it/s]

In [38]:
df['Fog Index'] = f_index
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length,Complex Word Count,Percentage of Complex words,Fog Index
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13,798,57.16,30.116
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230,80,12.65,397,39.23,20.752
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088,85,14.76,707,56.33,28.436
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163,95,12.18,513,44.34,22.608
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157,79,15.86,627,50.04,26.360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268,49,13.71,331,49.26,25.188
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107,62,19.55,559,46.12,26.268
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216,66,12.83,460,54.31,26.856
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337,29,17.79,308,59.69,30.992


##Words per sentense

In [39]:
word_per_sentense = []

for i, row in tqdm(df.iterrows(), total=len(df)):
  text = row['Context']
  total_sentense = sent_tokenize(text)
  sentense_count = len(total_sentense)
  
  words = []

  for sentense in total_sentense:
    for word in sentense:
      words.append(word)
  
  word_count = len(words)
  
  wps = round((word_count / sentense_count),2)
  word_per_sentense.append(wps)

  0%|          | 0/114 [00:00<?, ?it/s]

In [40]:
df['Average Number of Words Per Sentence'] = word_per_sentense
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length,Complex Word Count,Percentage of Complex words,Fog Index,Average Number of Words Per Sentence
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13,798,57.16,30.116,157.13
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230,80,12.65,397,39.23,20.752,104.96
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088,85,14.76,707,56.33,28.436,129.80
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163,95,12.18,513,44.34,22.608,101.79
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157,79,15.86,627,50.04,26.360,136.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268,49,13.71,331,49.26,25.188,120.00
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107,62,19.55,559,46.12,26.268,156.87
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216,66,12.83,460,54.31,26.856,108.02
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337,29,17.79,308,59.69,30.992,167.21


##Syllable per word

In [41]:
syllable_per_word = []

for i, row in tqdm(df.iterrows(), total=len(df)):
  text = row['Context']
  total_sentense = sent_tokenize(text)

  words = []
  sc = []

  for sentense in total_sentense:
    for word in sentense:
      words.append(word)

  for word in words:
    count = textstatistics().syllable_count(word)
    sc.append(count)
  
  spw = round((len(sc) / len(words)),2)
  syllable_per_word.append(spw)

  0%|          | 0/114 [00:00<?, ?it/s]

In [42]:
df['Syllable Count Per Word'] = syllable_per_word
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length,Complex Word Count,Percentage of Complex words,Fog Index,Average Number of Words Per Sentence,Syllable Count Per Word
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13,798,57.16,30.116,157.13,1.0
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230,80,12.65,397,39.23,20.752,104.96,1.0
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088,85,14.76,707,56.33,28.436,129.80,1.0
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163,95,12.18,513,44.34,22.608,101.79,1.0
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157,79,15.86,627,50.04,26.360,136.63,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268,49,13.71,331,49.26,25.188,120.00,1.0
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107,62,19.55,559,46.12,26.268,156.87,1.0
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216,66,12.83,460,54.31,26.856,108.02,1.0
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337,29,17.79,308,59.69,30.992,167.21,1.0


##Personal pronouns

In [43]:
import re

personal_pronouns_count = []

for i, row in tqdm(df.iterrows(), total=len(df)):
  text = row['Context']

  pronounRegex = re.compile(r'I|we|my|ours|us',re.I)
  pronouns = pronounRegex.findall(text)
  personal_pronouns_count.append(len(pronouns))

  0%|          | 0/114 [00:00<?, ?it/s]

In [44]:
df['Personal Pronouns'] = personal_pronouns_count
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length,Complex Word Count,Percentage of Complex words,Fog Index,Average Number of Words Per Sentence,Syllable Count Per Word,Personal Pronouns
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13,798,57.16,30.116,157.13,1.0,933
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230,80,12.65,397,39.23,20.752,104.96,1.0,600
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088,85,14.76,707,56.33,28.436,129.80,1.0,875
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163,95,12.18,513,44.34,22.608,101.79,1.0,684
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157,79,15.86,627,50.04,26.360,136.63,1.0,760
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268,49,13.71,331,49.26,25.188,120.00,1.0,402
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107,62,19.55,559,46.12,26.268,156.87,1.0,677
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216,66,12.83,460,54.31,26.856,108.02,1.0,518
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337,29,17.79,308,59.69,30.992,167.21,1.0,397


##Average word length

In [45]:
average_length = []

for i, row in tqdm(df.iterrows(), total=len(df)):
  text = row['Context']
  total_sentense = sent_tokenize(text)

  words = []
  sc = []

  for sentense in total_sentense:
    for word in sentense:
      words.append(word)

  average = round(sum(len(word) for word in words) / len(words),2) 
  average_length.append(average)

  0%|          | 0/114 [00:00<?, ?it/s]

In [46]:
df['Average Word Length'] = average_length
df

,Ids,neg,neu,pos,compound,Websites,Context,Count of words,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length,Complex Word Count,Percentage of Complex words,Fog Index,Average Number of Words Per Sentence,Syllable Count Per Word,Personal Pronouns,Average Word Length
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13,798,57.16,30.116,157.13,1.0,933,1.0
1,1,0.084,0.767,0.149,0.9984,https://insights.blackcoffer.com/what-if-the-c...,"Human minds, a fascination in itself carrying...",1012,0.278969,0.000230,80,12.65,397,39.23,20.752,104.96,1.0,600,1.0
2,2,0.036,0.889,0.075,0.9891,https://insights.blackcoffer.com/what-jobs-wil...,Introduction AI is rapidly evolving in the em...,1255,0.351348,0.000088,85,14.76,707,56.33,28.436,129.80,1.0,875,1.0
3,3,0.044,0.811,0.145,0.9995,https://insights.blackcoffer.com/will-machine-...,“Anything that could give rise to smarter-tha...,1157,0.534389,0.000163,95,12.18,513,44.34,22.608,101.79,1.0,684,1.0
4,4,0.050,0.804,0.147,0.9996,https://insights.blackcoffer.com/will-ai-repla...,“Machine intelligence is the last invention t...,1253,0.492383,0.000157,79,15.86,627,50.04,26.360,136.63,1.0,760,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,109,0.051,0.820,0.129,0.9973,https://insights.blackcoffer.com/blockchain-fo...,Reconciling with the financial realities of a...,672,0.433331,0.000268,49,13.71,331,49.26,25.188,120.00,1.0,402,1.0
110,110,0.020,0.871,0.110,0.9993,https://insights.blackcoffer.com/the-future-of...,What Is an Investment? An investment is a res...,1212,0.692302,0.000107,62,19.55,559,46.12,26.268,156.87,1.0,677,1.0
111,111,0.092,0.818,0.091,-0.9301,https://insights.blackcoffer.com/big-data-anal...,Quality and affordable healthcare is a vision...,847,-0.005464,0.000216,66,12.83,460,54.31,26.856,108.02,1.0,518,1.0
112,112,0.004,0.825,0.170,0.9989,https://insights.blackcoffer.com/business-anal...,Analytics is a statistical scientific process...,516,0.954018,0.000337,29,17.79,308,59.69,30.992,167.21,1.0,397,1.0


###Renaming columns

In [59]:
df = df.rename(columns={'neg':"Negative Score",'pos':"Positive Score",'Ids':"URL ID",'Websites':"URL",'Count of words':"Word Count",'Syllable Count Per Word':"Syllable Per Word"})

In [60]:
df.head(1)

,URL ID,Negative Score,neu,Positive Score,compound,URL,Context,Word Count,Polarity Score,Subjectivity Score,number_sentense,Average Sentense Length,Complex Word Count,Percentage of Complex words,Fog Index,Average Number of Words Per Sentence,Syllable Per Word,Personal Pronouns,Average Word Length
0,0,0.044,0.811,0.145,0.9996,https://insights.blackcoffer.com/ai-in-healthc...,Introduction “If anything kills over 10 milli...,1396,0.534389,0.000135,77,18.13,798,57.16,30.116,157.13,1.0,933,1.0


###Dropping unnecessary columns

In [62]:
df = df.drop(['neu','compound','Context','number_sentense'],axis = 1)
df.head(1)

,URL ID,Negative Score,Positive Score,URL,Word Count,Polarity Score,Subjectivity Score,Average Sentense Length,Complex Word Count,Percentage of Complex words,Fog Index,Average Number of Words Per Sentence,Syllable Per Word,Personal Pronouns,Average Word Length
0,0,0.044,0.145,https://insights.blackcoffer.com/ai-in-healthc...,1396,0.534389,0.000135,18.13,798,57.16,30.116,157.13,1.0,933,1.0


In [61]:
df.columns

Index(['URL ID', 'Negative Score', 'neu', 'Positive Score', 'compound', 'URL',
       'Context', 'Word Count', 'Polarity Score', 'Subjectivity Score',
       'number_sentense', 'Average Sentense Length', 'Complex Word Count',
       'Percentage of Complex words', 'Fog Index',
       'Average Number of Words Per Sentence', 'Syllable Per Word',
       'Personal Pronouns', 'Average Word Length'],
      dtype='object')

###Rearranging the columns

In [64]:
df  = df[['URL ID','URL','Positive Score','Negative Score','Polarity Score','Subjectivity Score','Average Sentense Length','Percentage of Complex words',
          'Fog Index','Average Number of Words Per Sentence','Complex Word Count','Word Count','Syllable Per Word','Personal Pronouns', 'Average Word Length']]

In [65]:
df.head(2)

,URL ID,URL,Positive Score,Negative Score,Polarity Score,Subjectivity Score,Average Sentense Length,Percentage of Complex words,Fog Index,Average Number of Words Per Sentence,Complex Word Count,Word Count,Syllable Per Word,Personal Pronouns,Average Word Length
0,0,https://insights.blackcoffer.com/ai-in-healthc...,0.145,0.044,0.534389,0.000135,18.13,57.16,30.116,157.13,798,1396,1.0,933,1.0
1,1,https://insights.blackcoffer.com/what-if-the-c...,0.149,0.084,0.278969,0.000230,12.65,39.23,20.752,104.96,397,1012,1.0,600,1.0


###Saving the Final Output to an excel file

In [66]:
df.to_excel('Final_Output.xlsx')